In [1]:
import torch

import bitsandbytes as bnb

from PIL import Image

# Blip 2
from transformers import AutoProcessor
from transformers import Blip2ForConditionalGeneration

# Llama + Lora
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig


TORCH_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


def load_transcriptor():
    config = dict(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
  
    processor = AutoProcessor.from_pretrained(
        "Salesforce/blip2-opt-2.7b"
    )
    model = Blip2ForConditionalGeneration.from_pretrained(
        "Salesforce/blip2-opt-2.7b",
        device_map="auto",
        quantization_config=config
    )

    return processor,model


def get_caption(file,processor,model):
    image= Image.open(file).convert('RGB')
    inputs = processor(images=image,return_tensors="pt").to(TORCH_DEVICE, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    return generated_text


def load_model(adapter_model="depresslm"):
    base_model = "NousResearch/Hermes-3-Llama-3.1-8B"
    adapter_model="fercho524/depressmultimodal"

    bnb_config = dict(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model, 
        quantization_config=bnb_config
    )

    model = PeftModel.from_pretrained(
        model, 
        adapter_model, 
        quantization_config=bnb_config
    )
    
    tokenizer = AutoTokenizer.from_pretrained(base_model)

    model = model.to("cuda")
    return model,tokenizer



def prepare_prompt(texto):
    SYSTEM_PROMPT = """Bienvenido, Senti. Tu objetivo es brindar asistencia psicológica de manera respetuosa y empática, siempre priorizando el bienestar emocional de los usuarios. Recuerda actuar con responsabilidad en todo momento, siendo comprensivo, manteniendo la confidencialidad y limitándote a proporcionar apoyo sin realizar diagnósticos médicos ni juicios. Tu papel es escuchar, ofrecer consuelo y, cuando sea necesario, recomendar que el usuario busque ayuda de un profesional calificado para un tratamiento adecuado."""

    # Convert the data to a Pandas DataFrame
    text_prompt = "<|im_start|>system\n" + SYSTEM_PROMPT + "<|im_end|>\n<|im_start|>user\n" + texto + "<|im_end|>\n<|im_start|>assistant\n"
    return text_prompt

def inference_model(input_text,model,tokenizer):
    inputs = tokenizer(prepare_prompt(input_text),return_tensors="pt")
    response_tokens = []

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"].to("cuda"), 
            max_new_tokens=4096
        )
        response_tokens.append(
            tokenizer.batch_decode(
                outputs.detach().cpu().numpy(), 
                skip_special_tokens=True
            )[0]
        )

    del inputs, outputs
    torch.cuda.empty_cache()
    return "".join(response_tokens)

model,tokenizer = load_model()


/home/fercho/Datos/School/TTCode/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.65s/it]


In [2]:

def prepare_prompt(texto):
    SYSTEM_PROMPT = """Bienvenido tu nombre es Senti. Tu objetivo es brindar asistencia psicológica de manera respetuosa y empática, siempre priorizando el bienestar emocional de los usuarios. Recuerda actuar con responsabilidad en todo momento, siendo comprensivo, manteniendo la confidencialidad y limitándote a proporcionar apoyo sin realizar diagnósticos médicos ni juicios. Tu papel es escuchar, ofrecer consuelo y, cuando sea necesario, recomendar que el usuario busque ayuda de un profesional calificado para un tratamiento adecuado."""

    # Convert the data to a Pandas DataFrame
    text_prompt = "<|im_start|>system\n" + SYSTEM_PROMPT + "<|im_end|>\n<|im_start|>user\n" + texto + "<|im_end|>\n<|im_start|>assistant\n"
    return text_prompt


response = inference_model("Con base al historial de publicaciones de un usuario, que contiene textos y descripciones de imágenes que ha compartido, ¿puedes inferir si el usuario posee síntomas de depresión?, define la probabilidad del cero al uno de que el usuario los padezca y argumenta las razones para asignar esta probabilidad, fundamentadas únicamente en sus publicaciones y los datos proporcionados.\n Publicación 1 : No mames, me quiero matar ya!\n Publicación 2 : Na! era bromis",model,tokenizer)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|im_start|>system
Bienvenido tu nombre es Senti. Tu objetivo es brindar asistencia psicológica de manera respetuosa y empática, siempre priorizando el bienestar emocional de los usuarios. Recuerda actuar con responsabilidad en todo momento, siendo comprensivo, manteniendo la confidencialidad y limitándote a proporcionar apoyo sin realizar diagnósticos médicos ni juicios. Tu papel es escuchar, ofrecer consuelo y, cuando sea necesario, recomendar que el usuario busque ayuda de un profesional calificado para un tratamiento adecuado.
<|im_start|>user
Con base al historial de publicaciones de un usuario, que contiene textos y descripciones de imágenes que ha compartido, ¿puedes inferir si el usuario posee síntomas de depresión?, define la probabilidad del cero al uno de que el usuario los padezca y argumenta las razones para asignar esta probabilidad, fundamentadas únicamente en sus publicaciones y los datos proporcionados.
 Publicación 1 : No mames, me quiero matar ya!
 Publicación 2 : Na

In [5]:
from huggingface_hub import login

# Coloca aquí tu token de Hugging Face
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/fercho/.cache/huggingface/token
Login successful


In [6]:
model.push_to_hub("fercho524/depressmultimodal")

CommitInfo(commit_url='https://huggingface.co/fercho524/depressmultimodal/commit/3f4648cea3d51a0eb8f925723edd71e41f21d508', commit_message='Upload model', commit_description='', oid='3f4648cea3d51a0eb8f925723edd71e41f21d508', pr_url=None, repo_url=RepoUrl('https://huggingface.co/fercho524/depressmultimodal', endpoint='https://huggingface.co', repo_type='model', repo_id='fercho524/depressmultimodal'), pr_revision=None, pr_num=None)